This Notebook explains how to pass dataset between scripts

In [1]:
#import os
#os.makedirs('script_exp_dfload')

In [50]:
%%writefile script_exp_dfload/script.py
from azureml.core import Run,Dataset
import pandas as pd
import argparse as arg

parser = arg.ArgumentParser()
parser.add_argument('--ds',type=str,dest='dataset_name')#receiving registered dataset name as parameter
parser.add_argument('--dsobj',type=str,dest='dataset_name2')#receiving registered dataset obj as parameter
args = parser.parse_args()
print('************************************')
print(args.dataset_name2)
print('************************************')
run = Run.get_context()
ws = run.experiment.workspace
dataset = Dataset.get_by_name(ws,name=args.dataset_name)#read data from 1st arg
dataset2 = Dataset.get_by_id(ws,id=args.dataset_name2) #read data from 2nd arg

print(dataset.take(3).to_pandas_dataframe())
print('******************************')
print(dataset2.take(3).to_pandas_dataframe())

Overwriting script_exp_dfload/script.py


In [51]:
from azureml.core import Workspace,Datastore,Dataset

In [52]:
ws = Workspace.from_config()
print(ws)

Workspace.create(name='mlws', subscription_id='d8f2241f-6180-41e3-a1de-149047238a9a', resource_group='mlrg')


In [53]:
#dsobj = ws.datasets.get('wine quality')


In [54]:
#creating Environment as yaml file
from azureml.core import Environment,conda_dependencies,ScriptRunConfig
env = Environment('my_env')
packages = conda_dependencies.CondaDependencies.create(conda_packages=['pip'],
                                        pip_packages=['azureml-defaults','pandas']
                                        )
env.python.conda_dependencies = packages

dsobj2 = ws.datasets.get('wine_quality')

script_config = ScriptRunConfig(source_directory="script_exp_dfload",
                                script="script.py",
                                arguments=['--ds','wine quality', 
                                            '--dsobj',dsobj2.as_named_input('wine_quality')], #registered dataset name,dataset obj as parameter
                                environment=env)

In [55]:
dsobj2.as_named_input('wine_quality')

In [56]:
experiment_name = 'disp_df_data'
from azureml.core import Experiment
from azureml.widgets import RunDetails
exp = Experiment(workspace= ws,name = experiment_name)
run = exp.submit(config=script_config)
RunDetails(run).show()
run.wait_for_completion()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

{'runId': 'disp_df_data_1647338259_8d4dde0c',
 'target': 'local',
 'status': 'Finalizing',
 'startTimeUtc': '2022-03-15T09:57:43.289469Z',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'local',
  'ContentSnapshotId': 'ed8057fd-3cf7-4f71-bde2-727997db1f5b',
  'azureml.git.repository_uri': 'https://github.com/sureshgorakala/AzureML.git',
  'mlflow.source.git.repoURL': 'https://github.com/sureshgorakala/AzureML.git',
  'azureml.git.branch': 'main',
  'mlflow.source.git.branch': 'main',
  'azureml.git.commit': '261c12fa50f64aa267289020576f609eacfe0eef',
  'mlflow.source.git.commit': '261c12fa50f64aa267289020576f609eacfe0eef',
  'azureml.git.dirty': 'True'},
 'inputDatasets': [{'dataset': {'id': '0a09541a-27da-4e78-b168-76c9a6962493'}, 'consumptionDetails': {'type': 'RunInput', 'inputName': 'wine_quality', 'mechanism': 'Direct'}}, {'dataset': {'id': '392cfa2e-f87c-4405-83ae-37b7dac56466'}, 'consumptionDetails': {'type': 'Reference'}}, {'dataset': {'id': '0a09541a-27da-4e78-